In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
!ls

ABC.ipynb  Khushii  test_alzheimers


In [0]:
import os
os.chdir('/content/gdrive/My Drive/Colab Notebooks/Alzheimer_classification')

In [0]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout,Flatten, Dense
from keras import applications

img_w,img_h = 150, 150

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'Khushii/data/train'
validation_data_dir = 'Khushii/data/validation'
nb_train_samples = 9899
nb_validation_samples = 800
epochs = 100
batch_size = 16

def save_bottleneck_features():
  datagen = ImageDataGenerator(rescale=1./255)
  
  model = applications.VGG16(include_top=False,weights='imagenet')
  
  generator = datagen.flow_from_directory(
              train_data_dir,
              target_size=(img_w, img_h),
              batch_size=batch_size,
              class_mode=None,
              shuffle=False)
  
  bottleneck_features_train = model.predict_generator(
    generator,nb_train_samples//batch_size)
  
  np.save(open('bottleneck_features_train.npy', 'wb'), bottleneck_features_train)
            
                
  
  generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_w, img_h),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
  
  bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
  np.save(open('bottleneck_features_validation.npy', 'wb'),
           bottleneck_features_validation)
  
  
  
  
  
  
def train_top_model():
  train_data = np.load(open('bottleneck_features_train.npy','rb'))
  train_labels = np.array(
        [0] * int(nb_train_samples / 2) + [1] * int(nb_train_samples / 2))

  validation_data = np.load(open('bottleneck_features_validation.npy','rb'))
  validation_labels = np.array(
       [0] * int(nb_validation_samples / 2) + [1] * int(nb_validation_samples / 2))
  model = Sequential()
  model.add(Flatten(input_shape=train_data.shape[1:]))
  model.add(Dense(256,activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1,activation='sigmoid'))
  
  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  
  model.fit(train_data,train_labels,
            epochs=epochs,
            batch_size=batch_size,
           validation_data=(validation_data,validation_labels))
  
  model.save_weights(top_model_weights_path)
  model.save('model.h5')
  model.save_weights('weights.h5')
  model.save('model.h5')
  model.save_weights('weights.h5')

save_bottleneck_features()
train_top_model()

Found 9899 images belonging to 2 classes.
